In [1]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("Lightricks/LTX-Video")

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image = pipe(prompt).images[0]

model_index.json:   0%|          | 0.00/412 [00:00<?, ?B/s]

c:\Users\Manjusha Kumari\.conda\envs\pyramid\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Manjusha Kumari\.cache\huggingface\hub\models--Lightricks--LTX-Video. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


AttributeError: module diffusers has no attribute LTXPipeline

In [1]:
import torch

def make_linear_nd(x):
    """
    Reshape the tensor to (batch_size, -1) suitable for a fully connected layer.
    
    Args:
        x (torch.Tensor): Input tensor of shape (batch_size, *).

    Returns:
        torch.Tensor: Reshaped tensor of shape (batch_size, -1).
    """
    return x.view(x.size(0), -1)


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DualConv3d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, bias=True):
        super(DualConv3d, self).__init__()
        kernel_size = kernel_size
        
        if kernel_size == (1, 1, 1):
            raise ValueError("Kernel size should be greater than 1. Use make_linear_nd instead.")
        
        weight1 = nn.Parameter(torch.randn(out_channels, in_channels, *kernel_size))
        bias1 = nn.Parameter(torch.randn(out_channels)) if bias else None
        stride1 = stride
        padding1 = padding
        dilation1 = (1, 1, 1)
        groups = 1

    def forward(self, x, use_conv3d=False):
        if kernel_size == (1, 1, 1):
            x = make_linear_nd(x)
            # Here, you would normally pass this reshaped tensor to a linear layer
            return x
        else:
            x = F.conv3d(x, weight1, bias1, stride1, padding1, dilation1, groups)
            return x

# Example usage
in_channels = 3
out_channels = 5
kernel_size = (3, 3, 3)
stride = (2, 2, 2)
padding = (1, 1, 1)

# Create an instance of DualConv3d
dual_conv3d = DualConv3d(in_channels, out_channels, kernel_size, stride, padding, bias=True)

# Example input tensor
test_input = torch.randn(1, 3, 10, 10, 10)

# Perform forward pass
output = dual_conv3d(test_input, use_conv3d=True)
print(output.shape)


torch.Size([1, 5, 5, 5, 5])


In [4]:
import math 
import torch 

# a = torch.sqrt(2)
# print(a)

b = math.sqrt(2)
print(b)

1.4142135623730951


In [62]:
import torch

def blend_v(
        a: torch.Tensor,
        b: torch.Tensor,
        blend_extent: int
) -> torch.Tensor:
    
    blend_extent = min(a.shape[3], b.shape[3], blend_extent)
    print("blend_extent", blend_extent)

    for y in range(blend_extent):
        print("y", y)
        print("object: -> ", b[:, :, :, y, :])
        print("calulate", -blend_extent + y)
        # print("blend extent2 ", a.size()) -> torch.Size([1, 1, 1, 4, 4])
        print("blend extend", blend_extent)
        print("blend_extent2: ", a[:, :, :, :, :])
        print("blend_extent3: ", a[:, :, :, -blend_extent + y, :])   # tensor([[[[2., 2., 2., 2.]]]])
        print("blend_extent4: ", a[:, :, :, -blend_extent + y, :] * (1 - y))  # y = 0 -> 1 - 0 = 1 But y = 1 -> 1 - 1 = 0
        print("blend_extent5: ", a[:, :, :, -blend_extent + y] * (1 - y / blend_extent))  # 1 / 2 = 1 and 0 / 2 = 0 
        # tensor([[[[2., 2., 2., 2.]]]]) + tensor([[[[5., 5., 5., 5.]]]])  =  tensor([[[[7., 7., 7., 7.]]]])
        print("blend_extent6: ", a[:, :, :, -blend_extent + y] * (1 - y / blend_extent) + b[:, :, :, y, :]) 
        print("multiple", b[:, :, :, y, :] * y) # 5 * 0 = 0 and 5 * 1 = 5 
        print("blend_extent7: ", a[:, :, :, -blend_extent + y] * (1 - y / blend_extent) + b[:, :, :, y, :] * y)  # 5 * 0 = 0 and 5 * 1 = 5 
        

        
        print(f"hello: {b[:, :, :, y, :]} = {a[:, :, :, -blend_extent + y, :]}")

    return b

# Create sample tensors
a = torch.ones((1, 1, 1, 4, 4)) * 2  # Tensor a filled with value 2
b = torch.ones((1, 1, 1, 4, 4)) * 5  # Tensor b filled with value 5

# Display original tensors
print("Original Tensor a:")
print(a)

print("Original Tensor b:")
print(b)

# Set blend_extent
blend_extent = 2

# Blend tensors vertically
blended_tensor = blend_v(a, b, blend_extent)

# Display blended tensor
print("Blended Tensor:")
print(blended_tensor)


Original Tensor a:
tensor([[[[[2., 2., 2., 2.],
           [2., 2., 2., 2.],
           [2., 2., 2., 2.],
           [2., 2., 2., 2.]]]]])
Original Tensor b:
tensor([[[[[5., 5., 5., 5.],
           [5., 5., 5., 5.],
           [5., 5., 5., 5.],
           [5., 5., 5., 5.]]]]])
blend_extent 2
y 0
object: ->  tensor([[[[5., 5., 5., 5.]]]])
calulate -2
blend extend 2
blend_extent2:  tensor([[[[[2., 2., 2., 2.],
           [2., 2., 2., 2.],
           [2., 2., 2., 2.],
           [2., 2., 2., 2.]]]]])
blend_extent3:  tensor([[[[2., 2., 2., 2.]]]])
blend_extent4:  tensor([[[[2., 2., 2., 2.]]]])
blend_extent5:  tensor([[[[2., 2., 2., 2.]]]])
blend_extent6:  tensor([[[[7., 7., 7., 7.]]]])
multiple tensor([[[[0., 0., 0., 0.]]]])
blend_extent7:  tensor([[[[2., 2., 2., 2.]]]])
hello: tensor([[[[5., 5., 5., 5.]]]]) = tensor([[[[2., 2., 2., 2.]]]])
y 1
object: ->  tensor([[[[5., 5., 5., 5.]]]])
calulate -1
blend extend 2
blend_extent2:  tensor([[[[[2., 2., 2., 2.],
           [2., 2., 2., 2.],
   

In [64]:
def blend_v(
            self,
            a: torch.Tensor,
            b: torch.Tensor,
            blend_extent: int
    ) -> torch.Tensor:
        
        blend_extent = min(a.shape[3],
                           b.shape[3],
                           blend_extent
                           )
        

        for y in range(blend_extent):
            b[:, :, :, y, :] = a[:, :, :, -blend_extent + y, :] * (
                1 - y / blend_extent
            ) + b[:, :, :, y, :] * (y / blend_extent)


        return b 
    

def blend_h(
            self,
            a: torch.Tensor,
            b: torch.Tensor,
            blend_extent:  int
    ) -> torch.Tensor:
        
        blend_extent = min(a.shape[4], b.shape[4], blend_extent)

        for x in range(blend_extent):
            b[:, :, :, :, x] = a[:, :, :, :, -blend_extent + x] * (
                1 - x / blend_extent
            ) + b[:, :, :, :, x] * (x / blend_extent)
    

        return b 

In [106]:
import torch
import torch.nn as nn

class EncoderDecoder:
    def __init__(self, tile_sample_min_size, tile_overlap_factor, tile_latent_min_size):
        self.tile_sample_min_size = tile_sample_min_size
        self.tile_overlap_factor = tile_overlap_factor
        self.tile_latent_min_size = tile_latent_min_size
        self.encoder = nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1)  # Dummy encoder
        self.quant_conv = nn.Conv2d(8, 8, kernel_size=1, stride=1, padding=0)  # Dummy quant conv

    def blend_h(self, left_tile, current_tile, blend_extent):
        # Dummy blend horizontal function
        return current_tile

    def blend_v(self, above_tile, current_tile, blend_extent):
        # Dummy blend vertical function
        return current_tile

    def _hw_tiled_encode(self, x: torch.FloatTensor, return_dict: bool = True):
        print("x", x)
        overlap_size = int(self.tile_sample_min_size * (1 - self.tile_overlap_factor))  
        print("overlap_size: ", overlap_size) # 3

        blend_extent = int(self.tile_latent_min_size * self.tile_overlap_factor)
        print("blend_extent: ", blend_extent)  # 1 


        row_limit = self.tile_latent_min_size - blend_extent
        print("row_limit: ", row_limit)  # torch.Size([1, 8, 4, 4])



        rows = []
        for i in range(0, x.shape[3], overlap_size):
            print("i: ", i)

            row = []
            for j in range(0, x.shape[4], overlap_size):
                print("j: ", j)
                
                tile = x[:, :, :, i: i + self.tile_sample_min_size, j: j + self.tile_sample_min_size]
                print("before tile: ", tile)
                print("before add tile: ", x[:, :, :, i:] )
                print("after add tile: ", x[:, :, :, i: + self.tile_sample_min_size, ])

                # Remove the extra dimension
                tile = tile.squeeze(2)
                print("After tile squeeze: ", tile)

                tile = self.encoder(tile)
                print("tile pass in the encoder: ", tile)

                tile = self.quant_conv(tile)
                print("tile pass in the quant_conv: ", tile)

                row.append(tile)
                print("row: ", row)

            rows.append(row)
            print("rows: ", rows)

        result_rows = []
        for i, row in enumerate(rows):
            print("i: ", i)
            print("row: ", row)


            result_row = []
            for j, tile in enumerate(row):
                print("j: ", j)
                print("tile: ", tile)

                if i > 0:
                    tile = self.blend_v(rows[i - 1][j], tile, blend_extent)
                    print("i is greater then 0: ", tile)

                if j > 0:
                    tile = self.blend_h(row[j - 1], tile, blend_extent)
                    print("j is greater then 0: ", tile)


                result_row.append(tile[:, :, :row_limit, :row_limit])
                print("result_row: ", result_row)

            result_rows.append(torch.cat(result_row, dim=3))

        moment = torch.cat(result_rows, dim=2)
        print("moment: ", moment)
        return moment


# Example usage
tile_sample_min_size = 4
tile_overlap_factor = 0.25
tile_latent_min_size = 4

encoder_decoder = EncoderDecoder(tile_sample_min_size, tile_overlap_factor, tile_latent_min_size)

# Creating a dummy tensor with shape [1, 3, 1, 4, 4] (batch size, channels, depth, height, width)
dummy_tensor = torch.randn(1, 3, 1, 4, 4)

# Encode the dummy tensor
encoded_tensor = encoder_decoder._hw_tiled_encode(dummy_tensor)

print(encoded_tensor.shape)  # Check the shape of the encoded tensor


x tensor([[[[[ 1.5201,  0.5186,  0.7231,  0.1764],
           [ 0.9649,  0.3002,  0.2164, -1.2856],
           [-0.0429, -0.9251, -1.8161, -1.5204],
           [-0.3214, -0.6163,  0.8686, -1.5710]]],


         [[[ 0.9377, -1.5524, -0.4212, -0.0374],
           [-1.1621, -0.7998,  1.3137,  0.9624],
           [-1.6743,  0.2641, -0.1526, -1.4286],
           [-1.4594, -1.3954, -0.1143,  0.5692]]],


         [[[ 1.3877,  0.3580,  0.1190,  0.0122],
           [ 1.0036,  0.4932, -0.4532,  1.2837],
           [ 0.2997,  1.6076, -1.2261, -0.8858],
           [-1.5739, -0.4865,  1.5061,  0.1685]]]]])
overlap_size:  3
blend_extent:  1
row_limit:  3
i:  0
j:  0
before tile:  tensor([[[[[ 1.5201,  0.5186,  0.7231,  0.1764],
           [ 0.9649,  0.3002,  0.2164, -1.2856],
           [-0.0429, -0.9251, -1.8161, -1.5204],
           [-0.3214, -0.6163,  0.8686, -1.5710]]],


         [[[ 0.9377, -1.5524, -0.4212, -0.0374],
           [-1.1621, -0.7998,  1.3137,  0.9624],
           [-1.6743,  0.26